In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import networkx as nx 
import pickle as pkl

import h5py

In [29]:
# gene id mapping
with open('../../../GENEID_MAPPING/data/processed/all2genesymbol.pkl', 'rb') as f: 
    all2symb = pkl.load(f)

geneinfo = pd.read_csv('../data/raw/geneinfo_beta.txt', sep='\t', dtype=str)
id2symb = geneinfo.set_index('gene_id')['gene_symbol'].to_dict()

expr_info = pd.read_csv('../data/raw/instinfo_beta.txt', sep='\t')
expr_info.head()

sampinfo = pd.read_csv('../data/raw/instinfo_beta.txt', '\t', dtype=str)


C:\Users\natha\miniconda3\envs\gcnvae\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
with open('../../../cell_baseline.pkl', 'rb') as f: 
    baseline_expr = pkl.load(f)
    
print('keys:', baseline_expr.keys())

cell_lines = baseline_expr['linc_ccle_lines']
len(cell_lines)

keys: dict_keys(['unpert_expr', 'linc_ccle_lines', 'meta', 'source', 'creation_date'])


138

In [12]:
with open('../../../DDS_graph.pkl', 'rb') as f: 
    DDS_graph = pkl.load(f)

with open('../../../PPI_graph.pkl', 'rb') as f: 
    PPI_graph = pkl.load(f)

In [13]:
drugs = DDS_graph['nodelist']
genes = PPI_graph['nodelist']

In [14]:
cp = h5py.File('../data/raw/level3_beta_trt_cp_n1805898x12328.gctx', 'r')

h5_data= cp['0']['DATA']['0']['matrix']

sample_meta = np.array(cp['0']['META']['COL']['id'], dtype=str)
gene_meta = np.array(cp['0']['META']['ROW']['id'], dtype=str)

In [16]:
gene_meta

array(['10', '100', '1000', ..., '9992', '9993', '9997'], dtype='<U12')

In [17]:
sample_meta

array(['ABY001_A375_XH_X1_B15:A13', 'ABY001_A375_XH_X1_B15:A14',
       'ABY001_A375_XH_X1_B15:A15', ..., 'TSAI002_NPC-8_XH_X1_B18:J16',
       'TSAI002_NPC-8_XH_X1_B18:J20', 'TSAI002_NPC-8_XH_X1_B18:J22'],
      dtype='<U47')

In [18]:
gene_meta_symb = np.array([all2symb[id2symb[x].upper()][0] if id2symb[x].upper() in all2symb else None for x in gene_meta])
len(gene_meta_symb)

12328

In [19]:
gene_selector = np.array([x in genes for x in gene_meta_symb])
np.sum(gene_selector)

9829

In [20]:
len(drugs)

1224

In [21]:
overlap_ = set(expr_info.cmap_name.str.lower().unique()).intersection(set(drugs))
len(overlap_)

1191

In [22]:
mydrug_obs = expr_info[lambda x: x.cmap_name.str.lower().isin(drugs)]
mydrug_obs.shape

(446874, 30)

In [23]:
mydrug_obs.head()

,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_itime,pert_time_unit,cell_mfc_name,pert_mfc_id,...,sample_id,pert_type,cell_iname,qc_pass,dyn_range,inv_level_10,build_name,failure_mode,project_code,cmap_name
7,f1b10,0.002,0.0015,uM,0.002 uM,6.0,6 h,h,A549,BRD-A61304759,...,HOG001_A549_6H_X4_F1B10:P11,trt_cp,A549,1.0,12.47150,2301.0,NaN,NaN,HOG,tanespimycin
28,b10,0.004,0.0045,uM,0.004 uM,24.0,24 h,h,MCF7,BRD-A61304759,...,HOG003_MCF7_24H_X1_B10:P10,trt_cp,MCF7,0.0,5.22482,6484.0,NaN,count_cv,HOG,tanespimycin
49,b1,15.000,17.1026,uM,15 uM,6.0,6 h,h,U2OS,BRD-A78377521-001-02-2,...,PAC005_U2OS_6H_X3_B1_UNI4445L:K10,trt_cp,U2OS,1.0,13.14520,3260.0,NaN,NaN,PAC,monastrol
51,b1,80.000,80.0000,uM,80 uM,24.0,24 h,h,HCC515,BRD-K57080016-001-04-3,...,CPC006_HCC515_24H_X5_B1_DUO52HI53LO:D04,trt_cp,HCC515,1.0,8.29863,2431.5,NaN,NaN,CPC,selumetinib
53,b10,0.004,0.0045,uM,0.004 uM,24.0,24 h,h,MCF7,BRD-A84481105,...,HOG001_MCF7_24H_X1_B10:G10,trt_cp,MCF7,1.0,5.13618,3903.5,NaN,NaN,HOG,thioridazine


In [ ]:
# what do we need from this? 

# 1 ) unperturbed expression (replicates?)
    # -> this is going to be in the `_ctl_` file. 
    # CHANGE - all unperturbed data will come from CCLE (expression [mutation, cnv, methyl])

# 2 ) perturbed expression (replicates? treat replicates as separate observations)

In [28]:
expr_info.pert_type.unique()

array(['trt_sh', 'ctl_vector', 'trt_lig', 'trt_oe', 'trt_cp', 'trt_aby',
       'trt_xpr', 'ctl_x', 'ctl_vehicle', 'ctl_untrt', 'trt_si'],
      dtype=object)

In [51]:
# does each cell line have unperturbed expression? and if so, are there replicates? 
    # no, only 62/? have untreated controls ... the vehicle controls have more tho -can we use those??
ctl_obs = expr_info[lambda x: (x.pert_type == 'ctl_untrt')]# & (x.pert_time == 0.)]

lines_w_ctrl = ctl_obs.cell_iname.unique()
ctl_obs.head()

,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_itime,pert_time_unit,cell_mfc_name,pert_mfc_id,...,sample_id,pert_type,cell_iname,qc_pass,dyn_range,inv_level_10,build_name,failure_mode,project_code,cmap_name
117138,b12,NaN,NaN,NaN,NaN,48.0,48 h,h,A375,CMAP-000,...,OEC001_A375_48H_X1.A2_B12:P09,ctl_untrt,A375,1.0,11.96920,2334.0,NaN,NaN,OEC,UnTrt
117151,f1b7,NaN,NaN,NaN,NaN,6.0,6 h,h,MCF7,CMAP-000,...,CRCGN004_MCF7_6H_X2_F1B7_DUO52HI53LO:B24,ctl_untrt,MCF7,0.0,4.60337,2594.0,NaN,dyn_range,CRCGN,UnTrt
117157,b11,NaN,NaN,NaN,NaN,120.0,120 h,h,PC3,CMAP-000,...,ERG013_PC3_120H_X3_B11:L12,ctl_untrt,PC3,1.0,7.80414,4144.0,NaN,NaN,ERG,UnTrt
117158,f1b7,NaN,NaN,NaN,NaN,6.0,6 h,h,MCF7,CMAP-000,...,CRCGN004_MCF7_6H_X2_F1B7_DUO52HI53LO:P03,ctl_untrt,MCF7,1.0,6.07042,2586.0,NaN,NaN,CRCGN,UnTrt
117161,f1b7,NaN,NaN,NaN,NaN,6.0,6 h,h,MCF7,CMAP-000,...,CRCGN004_MCF7_6H_X1_F1B7_DUO52HI53LO:M22,ctl_untrt,MCF7,1.0,6.33742,2845.5,NaN,NaN,CRCGN,UnTrt


In [25]:
# Using the cell lines in CCLE, how many obs do we have? 
mydruglines_obs = mydrug_obs[lambda x: x.cell_iname.isin(cell_lines.cell_iname.values)]
mydruglines_obs.cmap_name.unique().shape

(1189,)

In [26]:
# how many observations are there if we only use cell lines that have untreated controls? 
mydruglines_obs.shape # 300k + obs ... dropped ~100k 

(337790, 30)

In [28]:
mydruglines_obs.cell_iname.unique().shape

(132,)

In [33]:
# how many cell lines have vehcile controls? 
expr_info[lambda x: x.pert_type == 'ctl_vehicle'].cell_iname.unique().shape

(227,)

In [54]:
# how many cell lines have vehcile controls AND pert time == 0? could -666 (na val) mean 0 time? 
expr_info[lambda x: (x.pert_type == 'ctl_vehicle')].pert_time.unique()

array([  24.,    6.,   72.,    1.,    2.,    4.,    3.,   48., -666.,
         12.,  120.,   96.])

In [45]:
# how many replicates are there for each cell line (untreated controls)? 
ctl_obs.groupby('cell_iname').count()['cmap_name'].sort_values()

cell_iname
P2X2B2        6
MCH58         8
NCIH716       8
NAMEC8       10
HUH751       13
           ... 
HA1E       2421
A375       2669
A549       2976
MCF7       3353
PC3        3448
Name: cmap_name, Length: 62, dtype: int64

In [46]:
ctl_obs[lambda x: x.cell_iname == 'PC3']

,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_itime,pert_time_unit,cell_mfc_name,pert_mfc_id,...,sample_id,pert_type,cell_iname,qc_pass,dyn_range,inv_level_10,build_name,failure_mode,project_code,cmap_name
117157,b11,NaN,NaN,NaN,NaN,120.0,120 h,h,PC3,CMAP-000,...,ERG013_PC3_120H_X3_B11:L12,ctl_untrt,PC3,1.0,7.80414,4144.0,NaN,NaN,ERG,UnTrt
117173,b11,NaN,NaN,NaN,NaN,72.0,72 h,h,PC3,CMAP-000,...,ERG013_PC3_72H_X1_B11:P12,ctl_untrt,PC3,1.0,7.48322,3345.0,NaN,NaN,ERG,UnTrt
117268,b1,NaN,NaN,NaN,NaN,144.0,144 h,h,PC3,CMAP-000,...,KDA007_PC3_144H_X1_B1_DUO44HI45LO:B18,ctl_untrt,PC3,0.0,6.13288,5446.0,NaN,qc_iqr,KDA,UnTrt
117281,b1,NaN,NaN,NaN,NaN,144.0,144 h,h,PC3,CMAP-000,...,KDB002_PC3_144H_X3_B1_DUO53HI52LO:M10,ctl_untrt,PC3,1.0,12.87110,4344.0,NaN,NaN,KDB,UnTrt
117351,b1,NaN,NaN,NaN,NaN,144.0,144 h,h,PC3,CMAP-000,...,KDA004_PC3_144H_X1_B1_DUO45HI44LO:P21,ctl_untrt,PC3,1.0,4.87489,5474.5,NaN,NaN,KDA,UnTrt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275546,b37,NaN,NaN,Unknown,NaN,96.0,96 h,h,PC3.311B,CMAP-000,...,XPR027_PC3.311B_96H_X3_B37:P23,ctl_untrt,PC3,1.0,21.67350,3186.0,NaN,NaN,XPR,UnTrt
1276201,b40,NaN,NaN,Unknown,NaN,96.0,96 h,h,PC3.311B,CMAP-000,...,XPR032_PC3.311B_96H_X1_B40:H16,ctl_untrt,PC3,1.0,10.03410,4997.0,NaN,NaN,XPR,UnTrt
1276227,b40,NaN,NaN,Unknown,NaN,96.0,96 h,h,PC3.311B,CMAP-000,...,XPR030_PC3.311B_96H_X1_B40:H24,ctl_untrt,PC3,1.0,5.36529,5148.0,NaN,NaN,XPR,UnTrt
1276543,b37,NaN,NaN,Unknown,NaN,96.0,96 h,h,PC3.311B,CMAP-000,...,XPR027_PC3.311B_96H_X3_B37:E10,ctl_untrt,PC3,1.0,18.01040,3458.0,NaN,NaN,XPR,UnTrt


In [37]:
sampinfo[lambda x: x.pert_dose.isna()][lambda x: x.pert_time == '0'][lambda x: x.pert_type == 'ctl_untrt'].groupby('cell_iname').count()['sample_id']

cell_iname
HME       18
NAMEC8    10
Name: sample_id, dtype: int64

In [ ]:
cp = h5py.File('../data/raw/level5_beta_trt_cp_n720216x12328.gctx', 'r')
cp_data = cp['0']['DATA']['0']['matrix']

sample_meta = np.array(cp['0']['META']['COL']['id'], dtype=str)
gene_meta = np.array(cp['0']['META']['ROW']['id'], dtype=str)

gene_meta_symb = np.array([all2symb[id2symb[x].upper()][0] if id2symb[x].upper() in all2symb else None for x in gene_meta])

gene_selector = np.array([x in genes for x in gene_meta_symb])

overlap_ = set(siginfo.cmap_name.str.lower().unique()).intersection(set(drugs))

mydrugs = siginfo[lambda x: x.cmap_name.str.lower().isin(drugs) & (x.pert_time == 24.0) & (x.pert_dose == 10.)]


sample_selector = pd.Series(sample_meta).isin(mydrugs.sig_id.values).values 

mysamples_all_drugs = cp_data[sample_selector.nonzero()[0], :]

mysamples_mygenes = mysamples_all_drugs[:, gene_selector]

mydata = pd.DataFrame(mysamples_mygenes)
mydata.columns = gene_meta_symb[gene_selector]
mydata.index = sample_meta[sample_selector]
mydata = mydata.merge(siginfo, left_index=True, right_on='sig_id', how='left')

return mydata, gene_meta_symb[gene_selector]



